In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import sys
import os

In [7]:
cd ..

C:\Projects\python\recommender


In [8]:
from datasets import TabularDataset, TorchMovielen10k

In [9]:
movielen = TorchMovielen10k('./inputs/ml-100k/u.data', user_min=4, item_min=4)

2019-07-12 22:54:50,661 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in ./inputs/ml-100k/u.data
I0712 22:54:50.661692  8884 torch_movielen.py:45] Read dataset in ./inputs/ml-100k/u.data
2019-07-12 22:54:50,728 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
I0712 22:54:50.728486  8884 torch_movielen.py:49] Original user size: 943
2019-07-12 22:54:50,728 - C:\Projects\python\recommender\utils.py - INFO - Original item size: 1682
I0712 22:54:50.728758  8884 torch_movielen.py:50] Original item size: 1682
2019-07-12 22:54:50,732 - C:\Projects\python\recommender\utils.py - INFO - Filter user size: 943
I0712 22:54:50.732749  8884 torch_movielen.py:56] Filter user size: 943
2019-07-12 22:54:50,733 - C:\Projects\python\recommender\utils.py - INFO - Filter item size: 1413
I0712 22:54:50.733747  8884 torch_movielen.py:57] Filter item size: 1413
C:\Projects\python\recommender\datasets\torch_movielen.py:84: SettingWithCopyWarning: 
A value is trying to

In [14]:
train_dl = movielen.get_dataloader(dataset_type='train', batch_size=4, num_workers=0)

In [15]:
for batch in train_dl:
    print(batch)
    break

tensor([[819, 746,  27],
        [150, 227, 630],
        [696, 125, 579],
        [597, 242, 254]])
